In [1]:
import pandas as pd

data = pd.read_csv("../data/listings.csv")

/Users/liumukun/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data = data.drop(data[data["isSold"] == 0].index).reset_index(drop=True)

In [3]:
top_two = data.bids.apply(lambda x: eval(x)[:2])

In [4]:
def EBAY_INCREMENT(current):
    if current < 1:
        return 0.05
    elif current < 5:
        return 0.2
    elif current < 15:
        return 0.5
    elif current < 60:
        return 1
    elif current < 150:
        return 2
    elif current < 300:
        return 5
    elif current < 600:
        return 10
    elif current < 1500:
        return 20
    elif current < 3000:
        return 50
    else:
        return 100

In [5]:
def INTERVALS(current):
    if current < 1:
        return 1
    elif current < 5:
        return 5
    elif current < 15:
        return 15
    elif current < 60:
        return 60
    elif current < 150:
        return 150
    elif current < 300:
        return 300
    elif current < 600:
        return 600
    elif current < 1500:
        return 1500
    elif current < 3000:
        return 3000

In [6]:
def rule_used(bids):
    increment = EBAY_INCREMENT(bids[-1])
    if bids[0]-bids[-1] != increment:
        return 0
    else:
        return 1

data["ruleTriggered"] = top_two.apply(rule_used)
data["increment"] = data.bids.apply(lambda x: eval(x)[0]+EBAY_INCREMENT(eval(x)[0]))
data.increment = data.increment.astype(float)

In [7]:
from ast import literal_eval

data.bids = data.bids.apply(literal_eval)

In [8]:
new_data = data.explode("bids").reset_index(drop=True)

In [9]:
def get_order(row):
    _id = row.id
    bid = row.bids
    original_bids = data[data.id == _id].bids.iloc[0]
    return original_bids.index(bid)+1

new_data["nobid"] = new_data.apply(get_order, axis=1)

In [10]:
for _id in new_data["id"].unique():
    listing = new_data[new_data["id"] == _id]
    for index, count in zip(list(listing["nobid"].value_counts().index), listing["nobid"].value_counts().values):
        for i in range(count-1):
            for j in range(i+1):
                l = new_data[(new_data["id"] == _id) & (new_data["nobid"] == index)].index[0]
                new_data.iloc[l,-1] = new_data.iloc[l,-1] + j + 1

In [11]:
apple = new_data[new_data.apple == 1]
samsung = new_data[new_data.samsung == 1]
amazon = new_data[new_data.amazon == 1]
others = new_data[new_data.others == 1]

new_data.loc[apple.index,["bids","startingPrice", "price", "increment"]] = (apple[["bids","startingPrice", "price", "increment"]]/apple.bids.std())
new_data.loc[samsung.index,["bids","startingPrice", "price", "increment"]] = (samsung[["bids","startingPrice", "price", "increment"]]/samsung.bids.std())
new_data.loc[amazon.index,["bids","startingPrice", "price", "increment"]] = (amazon[["bids","startingPrice", "price", "increment"]]/amazon.bids.std())
new_data.loc[others.index,["bids","startingPrice", "price", "increment"]] = (others[["bids","startingPrice", "price", "increment"]]/others.bids.std())

In [12]:
new_data.to_csv("../data/bids.csv", index=False)